In [42]:
# import libraries
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import numpy as np
import time
import random


https://www.basketball-reference.com/leagues/NBA_2020_games.html
https://www.basketball-reference.com/boxscores/201910220TOR.html

In [43]:
list_html = 'https://www.basketball-reference.com/boxscores/201910220TOR.html'

In [44]:
# Getting all references from the homepage
# for link in soup.find_all("a"):
#     print("Inner Text: {}".format(link.text))
#     print("Title: {}".format(link.get("title")))
#     print("href: {}".format(link.get("href")))

# Functions to scrape one game's box scores

In [45]:
def table_scraper(box_table):
    rows = []
    for child in box_table.find_all("tr"):
        row = []
        for td in child:
            try:
                #print(td.text)
                row.append(td.text)
            except:
                continue
        if len(row) > 0:
            rows.append(row)

    player_id = []

    for ref_player in box_table.find_all('a'):
        player_id.append(ref_player.get('href'))

    df = pd.DataFrame(rows[2:], columns = rows[1])
    df['starter'] = [1] * df[df.Starters == 'Reserves'].index[0] + [0] * (len(df.index) - df[df.Starters == 'Reserves'].index[0])
    df = df.drop(df[df.Starters == 'Reserves'].index[0])
    df = df.drop(df[df.Starters == 'Team Totals'].index[0])
    df = df.rename(columns = {'Starters': 'players'})
    df['player_id'] = player_id
    return(df)

In [46]:
def game_information_collector(soup):
    global_inf = []
    # Team IDs and name:
    for i in soup.find_all('a', attrs = {'itemprop' : 'name'}):
        global_inf.append(i.text) #Team name
        global_inf.append(i.get('href')) #ID
    # Score
    gam_res = soup.find_all("div", attrs={'class' : 'scorebox'})
    for i in gam_res[0].find_all('div', attrs = {'class' : "score"}):
        global_inf.append(i.text)
    # date and location
    for i in soup.find('div', attrs = {'class' : 'scorebox_meta'}).find_all('div'):
            global_inf.append(i.text)
    return(global_inf)

In [47]:
def single_page_handler(list_html):
    result = requests.get(list_html)
    soup = BeautifulSoup(result.content)
    
    #Global information:
    global_inf = game_information_collector(soup)

    counter = 0
    table_needed = []
    # Getting the location of necessary tables in a list
    for table_id in soup.find_all('table'):
        if 'game-basic' in table_id.get('id'):
            table_needed.append(counter)
        counter = counter + 1
        
    # Extract home team's tables
    table_home = table_scraper(soup.find_all('table')[table_needed[1]])
    table_home['team'] = global_inf[2]
    table_home['team_id'] = global_inf[3]
    
    # Extract away team's table
    table_away = table_scraper(soup.find_all('table')[table_needed[0]])
    table_away['team'] = global_inf[0]
    table_away['team_id'] = global_inf[1]
    
    # Concatenate tables
    df = table_home.append(table_away, ignore_index = True)
    df['game_id'] = list_html.rsplit('/', 1)[-1]
    return df

In [48]:
df = single_page_handler(list_html)

In [49]:
list_html = 'https://www.basketball-reference.com/boxscores/202012220BRK.html'
df = single_page_handler(list_html)

# Collecting links for the games

In [50]:
box_scores_link = "https://www.basketball-reference.com/leagues/NBA_2021_games.html"
result = requests.get(box_scores_link)
soup = BeautifulSoup(result.content)

In [51]:
for i in soup.find_all('table'):
    print(i.get('id'))
    
len(soup.find_all('table'))

schedule


1

In [52]:
def box_score_table_scraper(soup):   
    rows = []
    for child in soup.find_all('table')[0].find_all("tr"):
        row = []
        counter = -1
        for td in child:
            counter = counter + 1

            try:
                if counter in [2, 6, 4]:
                    row.append(td.text)
                    row.append(td.find('a').get('href'))
                else:
                    row.append(td.text)
            except:
                continue
        if len(row) > 0:
            rows.append(row)
    return rows[1:]

In [62]:
def get_one_season_links(url, main_url):
    result = requests.get(url)
    soup = BeautifulSoup(result.content)
    
    href_season_months = []
    for link in soup.find('div', attrs={'class' : 'filter'}).find_all("a"):
         href_season_months.append(link.get("href"))
    
    array = []
    for href_months in href_season_months:
        #print(main_url + href_months)
        result = requests.get(main_url + href_months)
        soup = BeautifulSoup(result.content)
        
        array = array + box_score_table_scraper(soup)
    
    df = pd.DataFrame(array[1:], columns = ['date', 'hour', 'visitor', 'visitor_id','visitor_score'
                                  , 'home', 'home_id', 'home_score', 'box_score', 'box_score_link',
                                  'OT', 'att', 'note']).drop(columns = ['box_score', 'note'])
    df = df[(df.home_score != '')]
    return(df)

In [63]:
main_url = 'https://www.basketball-reference.com/'
url = "https://www.basketball-reference.com/leagues/NBA_2021_games.html"
df = get_one_season_links(url, main_url)

In [64]:
def get_one_season_data(url, main_url):
    box_score_df = get_one_season_links(url, main_url)
    
    stats_all_game = pd.DataFrame()
    for box_link in box_score_df.box_score_link:
        print(main_url + box_link)
        stats_per_game = single_page_handler(main_url + box_link)
        stats_all_game.append(stats_per_game, ignore_index = True)
    
    return(box_score_df, stats_all_game)

In [65]:
start_time = time.time()

box_df, game_stats = get_one_season_data(url, main_url)

end_time = time.time()
print('Execution time = %.6f seconds' % (end_time-start_time))

https://www.basketball-reference.com//boxscores/202012220LAL.html
https://www.basketball-reference.com//boxscores/202012230CLE.html
https://www.basketball-reference.com//boxscores/202012230IND.html
https://www.basketball-reference.com//boxscores/202012230ORL.html
https://www.basketball-reference.com//boxscores/202012230PHI.html
https://www.basketball-reference.com//boxscores/202012230TOR.html
https://www.basketball-reference.com//boxscores/202012230BOS.html
https://www.basketball-reference.com//boxscores/202012230CHI.html
https://www.basketball-reference.com//boxscores/202012230MEM.html
https://www.basketball-reference.com//boxscores/202012230MIN.html
https://www.basketball-reference.com//boxscores/202012230DEN.html


KeyboardInterrupt: 

In [70]:
single_page_handler('https://www.basketball-reference.com//boxscores/202012230DEN.html')

,players,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,BLK,TOV,PF,PTS,+/-,starter,player_id,team,team_id,game_id
0,Nikola Jokić,41:50,11,18,.611,1,2,.500,6,9,...,2,6,3,29,+5,1,/players/j/jokicni01.html,Denver Nuggets,/teams/DEN/2021.html,202012230DEN.html
1,Gary Harris,41:33,4,9,.444,0,4,.000,2,2,...,1,0,2,10,+5,1,/players/h/harriga01.html,Denver Nuggets,/teams/DEN/2021.html,202012230DEN.html
2,Paul Millsap,36:06,7,13,.538,2,5,.400,0,0,...,0,1,3,16,-6,1,/players/m/millspa01.html,Denver Nuggets,/teams/DEN/2021.html,202012230DEN.html
3,Jamal Murray,34:27,1,9,.111,0,5,.000,7,7,...,0,2,6,9,+3,1,/players/m/murraja01.html,Denver Nuggets,/teams/DEN/2021.html,202012230DEN.html
4,Michael Porter,29:36,9,15,.600,3,7,.429,3,3,...,3,2,3,24,0,1,/players/p/portemi01.html,Denver Nuggets,/teams/DEN/2021.html,202012230DEN.html
5,Will Barton,33:27,7,16,.438,1,3,.333,1,2,...,1,0,2,16,+1,0,/players/b/bartowi01.html,Denver Nuggets,/teams/DEN/2021.html,202012230DEN.html
6,Monte Morris,16:19,1,6,.167,0,1,.000,0,0,...,0,1,0,2,-7,0,/players/m/morrimo01.html,Denver Nuggets,/teams/DEN/2021.html,202012230DEN.html
7,PJ Dozier,14:39,3,5,.600,0,1,.000,1,1,...,2,1,1,7,-6,0,/players/d/doziepj01.html,Denver Nuggets,/teams/DEN/2021.html,202012230DEN.html
8,Isaiah Hartenstein,8:53,3,6,.500,0,0,,0,0,...,2,2,4,6,-1,0,/players/h/harteis01.html,Denver Nuggets,/teams/DEN/2021.html,202012230DEN.html
9,Facundo Campazzo,8:08,1,1,1.000,1,1,1.000,0,0,...,0,0,1,3,-4,0,/players/c/campafa01.html,Denver Nuggets,/teams/DEN/2021.html,202012230DEN.html


In [ ]:
single_page_handler